In [0]:
import great_expectations as ge
reload(ge)

In [0]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import numpy as np

In [0]:
df = ge.read_csv("./data/ipps/ipps_drg.csv")

In [0]:
df.head()

In [0]:
np.log(df[" Total Discharges "]).hist()

In [0]:
np.log(df[" Average Total Payments "].map(lambda x: float(x[1]))).hist()

In [0]:
df["ln_avg_total_payments"] = df[" Average Total Payments "].map(lambda x: np.log(float(x[1])))
df.ln_avg_total_payments.hist()

In [0]:
df.ln_avg_total_payments.value_counts()

In [0]:
df.shape

In [0]:
my_partition = ge.dataset.util.partition_data(df.ln_avg_total_payments)
# my_partition = ge.dataset.util.partition_data(df.ln_avg_total_payments, bins="uniform", n_bins=10)
print my_partition
print len(my_partition['partition'])
plt.plot(my_partition['partition'][:-1], my_partition['weights'])

In [0]:
# df.expect_column_numerical_distribution_to_be("ln_avg_total_payments", my_partition, sample_size=163065)
results = []
for s in range(1,10) + range(10,100,10):
    new_result = df.expect_column_numerical_distribution_to_be("ln_avg_total_payments", my_partition, sample_size=s)
    new_result['sample_size'] = s
    results.append(new_result)

import pandas as pd
result_df = pd.DataFrame(results)

In [0]:
plt.plot(result_df.sample_size, np.log(result_df.true_value))
plt.hlines(np.log(.05), 0,1000)

# plt.plot(result_df.sample_size, result_df.true_value)
# plt.hlines(0.05, 0, 1000)

result_df.head()

In [0]:
# df.expect_column_numerical_distribution_to_be("ln_avg_total_payments", my_partition, sample_size=163065)
results = []
for s in range(1,100):
    new_result = df.expect_column_numerical_distribution_to_be("ln_avg_total_payments", my_partition, sample_size=50)
    new_result['sample_size'] = s
    results.append(new_result)

import pandas as pd
result_df = pd.DataFrame(results)

In [0]:
result_df.head()
result_df.true_value.hist()
result_df.true_value.mean()

In [0]:
df.expect_column_kl_divergence_to_be("ln_avg_total_payments", my_partition)

In [0]:
df['ln_total_discharges'] = np.log(df[" Total Discharges "])
np.log(df[" Total Discharges "]).hist(bins=100)

In [0]:
discharge_partition = ge.dataset.util.partition_data(df.ln_total_discharges)
discharge_partition
plt.plot(discharge_partition['partition'][:-1], discharge_partition['weights'])

In [0]:
df.expect_column_numerical_distribution_to_be('ln_total_discharges', discharge_partition)

In [0]:
# df.expect_column_numerical_distribution_to_be("ln_avg_total_payments", my_partition, sample_size=163065)
results = []
for s in range(1,100):
    new_result = df.expect_column_numerical_distribution_to_be("ln_total_discharges", discharge_partition)
    new_result['sample_size'] = s
    results.append(new_result)

import pandas as pd
result_df = pd.DataFrame(results)
result_df.true_value.hist()

In [0]:
df["ln_total_discharge_offset"] = df.ln_total_discharges# + .01
results = []
for s in range(1,1000):
    new_result = df.expect_column_numerical_distribution_to_be("ln_total_discharge_offset", discharge_partition)
    new_result['sample_size'] = s
    results.append(new_result)

import pandas as pd
result_df = pd.DataFrame(results)
result_df.true_value.hist()

In [0]:
# my_partition = ge.dataset.util.categorical_partition_data(df.avg_total_payments)
# plt.bar(my_partition['partition'], my_partition['weights'])
# df.expect_column_frequency_distribution_to_be("avg_total_payments", my_partition)

In [0]:
import random
df["stuff"] = df.ln_avg_total_payments.map(lambda x: random.choice(list("ABCDE")))
df.stuff.value_counts()

In [0]:
my_partition = ge.dataset.util.categorical_partition_data(df.stuff)
print my_partition
# plt.bar(my_partition['partition'], my_partition['weights'])

In [0]:
df.expect_column_frequency_distribution_to_be("stuff", my_partition)

In [0]:
my_list = list("ABCDE"*50)+["C"]

df["other_stuff"] = df.ln_avg_total_payments.map(lambda x: random.choice(my_list))

In [0]:
df.expect_column_frequency_distribution_to_be("other_stuff", my_partition)

In [0]:
expected_series = pd.Series(my_partition['weights'], index=my_partition['partition'], name='expected') * (1.*100)

In [0]:
expected_series